# Atomorbitaler for hydrogenliknende atomer
Her skal vi se på atomorbitaler for hydrogenliknende atomer. Med "hydrogenliknende" atomer mener vi atomer som bare har ett elektron. **Målet vårt er å visualisere  noen orbitaler i 3D.** Først kommer litt tekst som forklarer hva som visualiseres, hvis du bare vil se orbitalene kan du hoppe over dette og bare kjøre koden.

**Merk** En god del av det som kommer under går utenfor det som er pensum i KJ1000!

## Introduksjon

For hydrogenliknende atomer kan vi løse [Schrödingerligningen](https://snl.no/schr%C3%B6dingerligningen) analytisk. I 3D er dette lettest hvis vi bruker [kulekoordinater](https://no.wikipedia.org/wiki/Kulekoordinater) i stedet for de "vanlige" kartesiske koordinatene ($x$, $y$, $z$). Vi kaller løsningene for Schrödingerligningen for bølgefunksjoner, og vi bruker ofte symbolet $\psi$ for de. Disse bølgefunksjonene tolkes lettest ved Borns sannsynlighetstolkning som sier at $| \psi |^2$ er knyttet til sannsynlighetsfordelingen for elektronet.

Bølgefunksjonen ($\psi _{n\ell m}$) som løser Schrödingerligningen er på formen,

$$ \psi _{n\ell m}=R_{n\ell }(r)\,Y_{\ell m}(\theta ,\phi ),$$

der $n$, $\ell$ og $m$ er kvantetallene, $r$ av avstanden mellom elektronet og kjernen, og $\theta$ og $\psi$ er vinklene i kulekoordinatsystemet. Her er $R_{n\ell }(r)$ en funksjon som bare avhenger av $r$ (defineres lenger ned), mens $Y_{\ell m}(\theta ,\phi )$ er de sfærisk harmoniske funksjonene (engelsk: [spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics)). $Y_{\ell m}(\theta ,\phi )$ beskriver hvordan bølgefunksjonen avhenger av vinklene i kulekoordinatsystemet. Vi skal ikke gå nærmere inn på hvordan $Y_{\ell m}(\theta ,\phi )$ ser ut. Heldigvis er de tilgjengelige som funksjoner i Python via [scipy.special.sph_harm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html). 
Kvantetallene har visse krav for at $\psi_{n \ell m}$ skal være en matematisk gyldig løsning av Schrödingerligningen. De er alle heltall og de må oppfylle,

$$\begin{eqnarray} n&=&1,2,3,4,\dots  \\ \ell &=& 0,1,2,\dots ,n-1 \\ m&=&-\ell ,-\ell +1,\ldots ,0,\ldots ,\ell -1,\ell  \end{eqnarray}$$

For å visualisere orbitaler, så kan vi velge $n$, $\ell$ og $m$ slik at vi får en gyldig løsning og beregne $\psi_{n \ell m}$ for en rekke posisjoner. Deretter kan vi plotte $\vert \psi \vert^2$ for å få et bilde av sannsynlighetstettheten. Før vi gjør det, skal vi se litt nærmere på den [radielle avhengigheten](#Radiell-avhengighet) og [vinkelavhengigheten](#Vinkelavhengighet). Nederst i dette dokumentet vil du finne eksempler på matematiske uttrykk for [1s- og 2-orbitaler](#1s--og-2s-orbitaler) og [2p-orbitaler](#2p-orbitaler).

## Radiell avhengighet

Funksjonen $R_{n\ell }(r)$ er gitt ved,

$$R_{n\ell }(r)={\sqrt {{\left({\frac {2Z}{na_{\mu }}}\right)}^{3}{\frac {(n-\ell -1)!}{2n(n+\ell )!}}}}\text{e}^{-Zr/{na_{\mu }}}\left({\frac {2Zr}{na_{\mu }}}\right)^{\ell }L_{n-\ell -1}^{(2\ell +1)}\left({\frac {2Zr}{na_{\mu }}}\right), $$

der $ L_{n-\ell -1}^{(2\ell +1)}$ er de generaliserte Laguerre-polynomene (engelsk: [Laguerre polynomials](https://en.wikipedia.org/wiki/Laguerre_polynomials)), $Z$ er antallet protoner i kjernen og $a_{\mu }$ er en konstant. Her skal vi heller ikke bry oss så mye med hvordan Laguerre-polynomene faktisk ser ut, men vi konstaterer at disse også er tilgjengelige som funksjoner i Python via [scipy.special.genlaguerre](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.genlaguerre.html).

For å forenkle ting litt, sier vi at $Z = 1$ og at vi har valgt enheter slik at $a_{\mu } = 1$. Videre definerer vi $\hat{r} = \frac{2r}{n}$. Da blir $R_{n\ell }(r)$ litt kortere:

$$R_{n\ell }(r)={\sqrt {{\left({\frac {2}{n}}\right)}^{3}{\frac {(n-\ell -1)!}{2n(n+\ell )!}}}}  \text{e}^{-\hat{r}/2} {\hat{r}}^{\ell }  L_{n-\ell -1}^{(2\ell +1)}({\hat{r}}) .$$

### Kode for evaluering (og plotting) av radiell avhengighet
Vi implementerer ligningen for funksjonen $R_{n\ell }(r)$ gitt over. Denne er ganske grei å visualisere
siden den bare avhenger av avstanden.
Vi begynner med å importere noen matematiske funksjoner og funksjoner for plotting:

In [ ]:
from scipy.special import sph_harm  # Sfæriske harmoniske funksjoner
from scipy.special import genlaguerre  # Generaliserte Laguerre polynomer
from math import factorial  # Funksjon for å regne ut fakultet
import numpy as np  # Bibliotek for å kunne jobbe med numeriske lister
import pyvista as pv  # For 3D-plotting
pv.set_plot_theme('document')
from matplotlib import pyplot as plt # For plotting:
plt.style.use(['seaborn-notebook', '../kj1000.mplstyle'])
%matplotlib notebook

In [ ]:
def radiell(n, l, r):
    """Beregn radiell del av bølgefunksjonen
    
    Parametere
    ----------
    n : heltall, dette er hovedkvantetallet.
    l : heltall, dette er vinkelkvantetallet.
    r : tall, detter er posisjonen vi evaluerer funksjonen i
    
    Resultat
    --------
    ut : tall, verdien for radiell del i angitt punkt.
    
    """
    pre = np.sqrt((2 / n)**3 * factorial(n - l - 1) / (2 * n * factorial(n + l)))
    r_hat = 2 * r / n
    laguerre = genlaguerre(n - l - 1, 2*l + 1)
    return pre * np.exp(-r_hat / 2) * r_hat**l * laguerre(r_hat)

Over sa vi at $| \psi |^2$ kan tolkes som en sannsynlighet. Hvis vi har et lite volumelement, $\text{d} V$, så er
$\text{d} V \, | \psi |^2$ sannsynligheten for å finne elektronet i volumelementet. I kulekoordinater er $\text{d} V = r^2 \sin \theta \, \text{d} r \, \text{d} \theta \, \text{d} \phi$, men siden vi bare ser på radiell avhengighet, så kan vi tenke oss at vinkelavhengigheten er integrert bort, slik at det holder å se på $r^2 | \psi |^2$. La os plotte dette for noen kombinasjoner av kvantetall:

In [ ]:
# La oss undersøke radiell del for 1s, 2s og 2p:
r = np.linspace(0, 20, 1000)
fig, axi = plt.subplots(constrained_layout=True)
psi_r_1s = radiell(1, 0, r)
psi_r_2s = radiell(2, 0, r)
psi_r_2p = radiell(2, 1, r)

axi.plot(r, r**2 * abs(psi_r_1s)**2, label='1s', lw=3, alpha=0.8)
axi.plot(r, r**2 * abs(psi_r_2s)**2, label='2s', lw=3, alpha=0.8)
axi.plot(r, r**2 * abs(psi_r_2p)**2, label='2p', lw=3, alpha=0.8)
axi.set(xlabel='$\hat{r}$', ylabel='$\hat{r}^2 \\times |R_{n\ell}(\hat{r})|^2$', title='Radiell avhengighet')
axi.legend();

Denne figuren burde bli ganske lik figur 7.24 i læreboken på side 237.
Her merker vi oss at for høyere $n$, så kommer maksimalverdien lengre ut fra kjernen (større $\hat{r}$).

Vi kan tolke integralet av $\hat{r}^2 \vert \psi \vert^2$ fra $0$ opp til en gitt verdi for $\hat{r}$ som sannsynligheten for å finne elektronet et sted mellom $0$ og $\hat{r}$. 
La oss også plotte dette - integralet av av $\hat{r}^2 \vert \psi \vert^2$ fra $0$ opp til en gitt verdi for $\hat{r}$ kan vi regne ut numerisk ved å bruke funksjonen [cumulative_trapezoid](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumulative_trapezoid.html) fra SciPy:

In [ ]:
from scipy.integrate import cumulative_trapezoid  # Funksjon for numerisk integrasjon
# Integrasjon:
integ_1s = cumulative_trapezoid(r**2 * abs(psi_r_1s)**2, r)  # 1s
integ_2s = cumulative_trapezoid(r**2 * abs(psi_r_2s)**2, r)  # 2s
integ_2p = cumulative_trapezoid(r**2 * abs(psi_r_2p)**2, r)  # 2p

figi, axi = plt.subplots(constrained_layout=True)
axi.plot(r[1:], integ_1s, lw=3, alpha=0.8, label='1s')
axi.plot(r[1:], integ_2s, lw=3, alpha=0.8, label='2s')
axi.plot(r[1:], integ_2p, lw=3, alpha=0.8, label='2p')
axi.set(xlabel='$\hat{r}$', ylabel='$\int_{0}^\hat{r} \, r^2 \, |R_{n\ell} (r)|^2 \, \mathrm{d} r$',
        title='Sannsynlighetsfordelinger (funksjon av avstand)')
axi.legend();

Her ser vi at sannsynligheten er mye større nærmere kjernen for elektroner i 1s-orbitaler, og at sannsynligheten går mot 100 % (1 i plottet) når $\hat{r}$ øker. Vi kan prøve å kvantisere dette litt mer. F.eks. kan vi finne posisjonen der sannsynligheten er minst 90 %. La oss lage en generell funksjon som gjør dette for oss:

In [ ]:
def finn_posisjon(n, l, verdi=0.9):
    """Finner posisjon gitt en sannsynlighet.
    
    For en gitt sannsynlighet, finner vi den minste avstanden som er
    slik at sannsynligheten for å observere elektronet innen denne
    avstanden er (minst) lik den gitte sannsynligheten.
    
    Parametere
    ----------
    n : heltall, dette er hovedkvantetallet.
    l : heltall, dette er vinkelkvantetallet.
    verdi : tall, sannsynligheten vi ønsker.
    
    Returnerer
    ----------
    ut : tall, posisjonen som beskrevet over.
    
    """
    r = np.linspace(0, 20, 1000)
    funksjon = radiell(n, l, r)
    integral = cumulative_trapezoid(r**2 * abs(funksjon)**2, r)
    idx = np.argmax(integral > verdi)
    return r[idx + 1]

# La oss sjekke for 1s, 2s og 2p:
hvor_1s = finn_posisjon(1, 0, verdi=0.9)
hvor_2s = finn_posisjon(2, 0, verdi=0.9)
hvor_2p = finn_posisjon(2, 1, verdi=0.9)
print(f'1s, sannsynlighet > 0.9 for r >= {hvor_1s:.4g}')
print(f'2s, sannsynlighet > 0.9 for r >= {hvor_2s:.4g}')
print(f'2p, sannsynlighet > 0.9 for r >= {hvor_2p:.4g}')

figi, axi = plt.subplots(constrained_layout=True)
axi.plot(r[1:], integ_1s, lw=3, alpha=0.8, label='1s')
axi.plot(r[1:], integ_2s, lw=3, alpha=0.8, label='2s')
axi.plot(r[1:], integ_2p, lw=3, alpha=0.8, label='2p')
axi.axvline(x=hvor_1s, ls=':', color='k')
axi.axvline(x=hvor_2s, ls=':', color='k')
axi.axvline(x=hvor_2p, ls=':', color='k')
axi.axhline(y=0.9, ls=':', color='k')
axi.set(xlabel='$\hat{r}$', ylabel='$\int_{0}^\hat{r} \, r^2 \, R_{n\ell} (r) \, \mathrm{d} r$',
        title='Sannsynlighetsfordelinger (funksjon av avstand)')
axi.legend();

## Vinkelavhengighet
Vi hadde at $\psi _{n\ell m}=R_{n\ell }(r)\,Y_{\ell m}(\theta ,\phi )$, og vi har nettopp undersøkt $R_{n\ell }(r)$. La oss undersøke $Y_{\ell m}(\theta ,\phi)$.

Vi skal ikke si så mye om hvordan funksjonene $Y_{\ell m}(\theta ,\phi )$ ser ut. Men vi må påpeke en egenskap ved de som gjør visualisering litt mer komplekst, og det er at de kan være [komplekse funksjoner](https://no.wikipedia.org/wiki/Kompleks_analyse#Komplekse_funksjoner)! Disse funksjonene er komplekse når $m \neq 0$, men hvordan skal vi visualisere noe som kan være et komplekst tall? Her skal vi løse dette problemet ved å bruke en egenskap  som Schrödingerligningen har: Dersom vi har to løsninger $\psi_{n_1 \ell_1 m_1}$ og $\psi_{n_2 \ell_2 m_2}$, så er også summen av disse to, $\psi = a\psi_{n_1 \ell_1 m_1} + b\psi_{n_2 \ell_2 m_2}$, en gyldig løsning (her er $a$ og $b$ to tall som sikrer at $\psi$ er [normalisert](https://en.wikipedia.org/wiki/Normalizing_constant)). Ved å gjøre en passende sum, så kan vi få at $\psi$ er en reell funksjon, selv om både $\psi_{n_1 \ell_1 m_1}$ og $\psi_{n_2 \ell_2 m_2}$ er komplekse!

For å gjøre det hele litt mer konkret. La oss si at vi skal visualisere 2p-orbitalene. Da har vi tre mulige løsninger på Schrödingerligningen: $\psi_{2, 1, 0}$, $\psi_{2, 1, -1}$ og $\psi_{2, 1, 1}$. Den første funksjonen er reell ($m=0$), og det er denne vi vanligvis kaller for 2p$_\text{z}$,

$$\text{2p}_\text{z} = \psi_{2, 1, 0}.$$

De to andre er komplekse. og for å få reelle bølgefunksjoner sier vi heller at (merk at dette er et *valg* vi gjør!):

$$
\begin{eqnarray}
\text{2p}_\text{x}&=& \frac{1}{\sqrt{2}} (\psi_{2, 1, -1} - \psi_{2, 1, 1}), \\ 
\text{2p}_\text{y}&=& \frac{\text{i}}{\sqrt{2}} (\psi_{2, 1, -1} + \psi_{2, 1, 1}).
\end{eqnarray}$$

Generelt, kan vi gjøre følgende for å håndtere de komplekse funksjonene:
- Dersom $m=0$: Vi trenger ikke gjøre noe, vinkelfunksjonen er reell.
- Dersom $m<0$: Vi tar en sum på formen:
  $$\psi_{\text{orbital}} = \frac{\text{i}}{\sqrt{2}} \left(\psi_{n, \ell, m} - (-1)^m \psi_{n, \ell, -m}\right).$$
- Dersom $m > 0$: Vi tar en sum på formen:
  $$\psi_{\text{orbital}} = \frac{1}{\sqrt{2}} \left(\psi_{n, \ell, -m} + (-1)^m \psi_{n, \ell, m}\right).$$
  
### Kode for evaluering av vinkelavhengighet
Vinkelavhengigheten er litt vanskeligere å visualisere siden den avhenger av to variable: $\theta$ og $\psi$. Funksjoner av mer enn en variabel dekkes først i senere matematikkfag som f.eks. [MA0002 - Brukerkurs i matematikk B](https://www.ntnu.no/studier/emner/MA0002#tab=omEmnet). Men la oss likevel lage noe figurer som viser vinkelavhengigheten.

Vinkelavhengigheten kan vi regne ut vet å bruke de sfæriske harmoniske funksjonene fra SciPy. [SciPy-metoden](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html) bruker en litt annen notasjon og bytter om på vinklene, sammenliknet med hva vi har kalt de. Dette tar vi hensyn til når vi bruker funksjonen.

In [ ]:
def vinkelavhengighet(l, m, theta, phi):
    """Regn ut vinkelavhengighet for bølgefunksjonen.
    
    Her tar vi også hensyn til komplekse tall og gjør løsningene
    reelle.
    
    Parametere
    ----------
    l : heltall, vinkelkvantetallet.
    m : heltall, magnetisk kvantetall.
    theta : tall, polarvinkel (engelsk: polar angle).
    phi : tall, asimut (engelsk: azimuthal angle)
    
    Returnerer
    ----------
    ut : tall, verdi for vinkelavhengigheten til bølgefunksjonen.
    
    """
    # SciPy vil ha:
    # - m som første argument, l som andre
    # - asimut som første vinkel (kaller den "theta" i dokumentasjonen, dette blir vår "phi")
    # - polar som andre vinkel (kaller den "phi" i dokumentasjonen, dette blir vår "theta")
    if m == 0:
        vinkel = sph_harm(m, l, phi, theta)
    elif m < 0:
        vinkel = sph_harm(m, l, phi, theta) - (-1)**m * sph_harm(-m, l, phi, theta)
        vinkel = vinkel * (1j / np.sqrt(2))
    elif m > 0:
        vinkel = sph_harm(-m, l, phi, theta) + (-1)**m * sph_harm(m, l, phi, theta)
        vinkel = vinkel * (1 / np.sqrt(2))
    return np.real(vinkel)

Funksjonen over gjør egentlig ikke så mye. Den bare kaller på en annen funksjon, og passer på at input er konsistent med hvordan vi har definert ting. Vi kan nå prøve å plotte noen av de reelle sfæriske harmoniske. Først lager vi en funksjon som håndterer plottingen:

In [ ]:
def plot_hjelp(theta, phi, vinkel):
    """Lag et 3D plot ved å bruke PyVista, fargelegg etter verdi på vinkel"""
    # Det lettest å plotte om vi konverterer til kartesiske koordinater:
    xyz = np.array(
        [
            np.sin(theta) * np.cos(phi),
            np.sin(theta) * np.sin(phi),
            np.cos(theta),
        ]
    )
    X, Y, Z = np.abs(vinkel) * xyz
    grid = pv.StructuredGrid(X, Y, Z)
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('white')
    plotter.add_mesh(grid, scalars=vinkel.T, show_scalar_bar=False)
    plotter.show(jupyter_backend='ipygany')

La oss bruke denne funksjonen og lage noe plott for verdier av l og m, for å se hvordan funksjonene ser ut:

In [ ]:
# Lag interaktivt plott for l = 1, m = 0:
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)

plot_hjelp(theta, phi, vinkelavhengighet(1, 0, theta, phi))  # Fargene er her verdien på vinkelfunksjonen

In [ ]:
# Lag interaktivt plot for l=2, m=0:
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)

plot_hjelp(theta, phi, vinkelavhengighet(2, 0, theta, phi))  # Fargene er her verdien på vinkelfunksjonen

In [ ]:
# Interaktivt plot for l=3, m=0:
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)

plot_hjelp(theta, phi, vinkelavhengighet(3, 0, theta, phi))  # Fargene er her verdien på vinkelfunksjonen

In [ ]:
# Interaktivt plot for l=3, m=-2:
theta = np.linspace(0, np.pi, 250)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 250)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)

plot_hjelp(theta, phi, vinkelavhengighet(3, -2, theta, phi))  # Fargene er her verdien på vinkelfunksjonen

In [ ]:
# Lag statisk plot l = 1, 2, 3 og alle mulige m-verdier:

theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)
xyz = np.array(
    [
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta),
    ]
)
for l in (1, 2, 3):
    plotter = pv.Plotter(notebook=True, shape=(1, 2*l + 1), window_size=(1024, 200))

    for i, m in enumerate(range(-l, l+1)):
        plotter.subplot(0, i)
        plotter.add_text(f'l = {l}, m = {m}', font_size=10, color='k')
        vinkel = vinkelavhengighet(l, m, theta, phi)
        X, Y, Z = np.abs(vinkel) * xyz
        grid = pv.StructuredGrid(X, Y, Z)
        plotter.add_mesh(grid, scalars=vinkel.T, show_scalar_bar=False)
    plotter.show(jupyter_backend='static')

Jeg syns funksjonene over er ganske fine! Og de har former som kanskje minner om orbitaler? Det er ikke tilfeldig - det er i hovedsak vinkelfunksjonene som bestemmer formen. La oss oppsummere det vi har sett så langt:

* Vi har et matematisk uttrykk for bølgefunksjonen:
  - Vi har et uttrykk for radiell avhengighet (vi bruker en SciPy-funksjon for å beregne Laguerre-polynomer).
  - Vi har et uttrykk for vinkelavhengighet (vi bruker en SciPy-funksjon for å finne de sfærisk harmoniske funksjonene).
* Vi tar hensyn til at bølgefunksjonene kan være komplekse funksjoner. Vi "fikser" dette vet å ta summer av bølgefunksjoner for de tilfellene der $m \neq 0$.

Nå er vi klare til å visualisere orbitaler!

## Kode for å visualisere orbitaler
Nå kombinerer vi funksjonen for radiell avhengighet med funksjonen for vinkelavhengighet. Vi kan samtidig legge inn en sjekk som ser etter om kvantetallene vi putter inn er gyldige:

In [ ]:
def beregn_orbital(n, l, m, r, theta, phi):
    """Beregn bølgefunksjon ved å kombinere radiell avhengighet med vinkelavhengiget.
    
    Her sjekker vi også at kvantetallene som gis inn er gyldige.
    """
    # Sjekk at kvantetall er gyldige:
    if n < 1:
        raise ValueError(f'Ugyldig "n={n}". n = 1, 2, 3, ...')
    if l < 0 or l > n - 1:
        raise ValueError(f'Ugyldig "l={l}", l = 0, 1, ..., n-1')
    if m > l or m < -l:
        raise ValueError(f'Ugyldig "m={m}", m = -l, -l + 1, ..., 0, ..., l - 1, l')
    return radiell(n, l, r) * vinkelavhengighet(l, m, theta, phi)

Før vi plotter, lager vi en ny metode som kan hjelpe oss med plottingen. Denne vil fargelegge orbitalene etter verdien på bølgefunksjonen. Dette gjør det mulig for oss å se hvor bølgefunksjonen er positiv/negativ.

In [ ]:
def plot_hjelp_orbital(r, theta, phi, psi, cmap='viridis'):
    """Lag et 3D plot ved å bruke PyVista. Overflaten fargelegges etter verdiene til psi."""
    # Det lettest å plotte om vi konverterer til kartesiske koordinater:
    xyz = np.array(
        [
            r * np.sin(theta) * np.cos(phi),
            r * np.sin(theta) * np.sin(phi),
            r * np.cos(theta),
        ]
    )
    X, Y, Z = np.abs(psi)**2 * xyz
    grid = pv.StructuredGrid(X, Y, Z)
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('white')
    fortegn = 2. * (psi - psi.min()) / np.ptp(psi) - 1  # Skaler verdier for psi til [-1, 1]
    plotter.add_mesh(grid, scalars=fortegn.T, show_scalar_bar=True, cmap=cmap, clim=[-1, 1],
                     scalar_bar_args={'title': 'Skalert fortegn', 'color': 'k'})
    plotter.show(jupyter_backend='ipygany')

In [ ]:
# Lag et plot for n = 2, l = 1, m = 1. Dette burde bli et 2p-orbital
r = 5
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige r, theta og phi-kombinasjoner:
r, theta, phi = np.meshgrid(r, theta, phi)

psi_2p = beregn_orbital(2, 1, 1, r, theta, phi)
plot_hjelp_orbital(r, theta, phi, psi_2p)

In [ ]:
# Lag et plot for n = 3, l = 2, m = 0. Dette burde bli et 3d-orbital.
# Formen kan sammenliknes med figur 7.18 på side 232 i læreboken (men her plotter vi ikke helt det samme). 
r = 5
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige r, theta og phi-kombinasjoner:
r, theta, phi = np.meshgrid(r, theta, phi)

psi_3dz2 = beregn_orbital(3, 2, 0, r, theta, phi)
plot_hjelp_orbital(r, theta, phi, psi_3dz2)

In [ ]:
# Lag statisk plot for 2p- og 3d-orbitaler:

theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)
xyz = np.array(
    [
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta),
    ]
)
r = 3
for n, l in zip((2, 3), (1, 2)):
    plotter = pv.Plotter(notebook=True, shape=(1, 2*l + 1), window_size=(1000, 250))
    for i, m in enumerate(range(-l, l+1)):
        plotter.subplot(0, i)
        plotter.add_text(f'n = {n}, l = {l}, m = {m}', font_size=10, color='k')
        psi = beregn_orbital(n, l, m, r, theta, phi)
        orbital = np.abs(psi)**2
        X, Y, Z = orbital * xyz
        grid = pv.StructuredGrid(X, Y, Z)
        plotter.add_mesh(grid, scalars=psi.T, show_scalar_bar=False)
    plotter.show(jupyter_backend='static')

## Plotting av sannsynligheter
Over har vi plottet orbitaler ved å velge ut en bestemt verdi for $\hat{r}$ og så regner vi ut verdier for bølgefunksjonen for alle mulige $\theta$ og $\phi$. Ofte ønsker vi å lage figurer av orbitaler som er slik at det f.eks. er 90% sannsynlig at elektronet er inne i orbitalet. Videre, er det vanskelig å se noder (områder der $\psi = 0$) med plottene vi har nå.

For å gjøre det hele mer konkret, la oss si at vi skal visualisere 3s-orbitalet og vi ønsker å vise et volum slik at det er 90% sannsynlig å finne elektronet innen dette volumet. Innenfor dette volumet ønsker vi å vise verdien på bølgefunksjonen slik at vi kan se områder der sannsynligheten er 0.

Først finner vi hvor stor $\hat{r}$ vi må ha for å garantere 90% sannsynlighet:

In [ ]:
hvor_3s = finn_posisjon(3, 0, verdi=0.9)
print(f'r >= {hvor_3s}')

Så beregner vi noen overflater for utvalgte verdier av $\hat{r}$ opp til verdien vi fant over:

In [ ]:
plotter = pv.Plotter()

r = np.arange(0, 19.5 + 0.5, 0.5)
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(np.pi, 2*np.pi, 100)  # 180 <= phi <= 360, dette blir en halvkule
r, theta, phi = np.meshgrid(r, theta, phi)

# Beregn overflater:
xyz = np.array(
    [
        r * np.sin(theta) * np.cos(phi),
        r * np.sin(theta) * np.sin(phi),
        r * np.cos(theta),
    ]
)
psi = beregn_orbital(3, 0, 0, r, theta, phi)
orbital = np.abs(psi)**2
X, Y, Z = xyz
grid = pv.StructuredGrid(X, Y, Z)
plotter.add_mesh(grid, scalars=(r**2 * orbital).T,
                 show_scalar_bar=True,
                 scalar_bar_args={'title': 'Radiell sannsynlighet (r²ψ)', 'color': 'k'})

plotter.show(jupyter_backend='ipygany')

Fra figuren over kan vi se at vi har to områder med lav sannsynlighet (mørk farge) og et område med høyere sannsynlighet (i gult) relativt langt ut fra kjernen. Vi kan også gjøre en sammenlikning for 1s, 2s og 3s:

In [ ]:
plotter = pv.Plotter()

r = np.arange(0., 20, 0.05)
theta = np.linspace(0, np.pi, 50)  # 0 <= theta <= 180
phi = np.linspace(np.pi, 2*np.pi, 50)  # 180 <= phi <= 360, dette blir en halvkule
r, theta, phi = np.meshgrid(r, theta, phi)

# Beregn overflater:
xyz = np.array(
    [
        r * np.sin(theta) * np.cos(phi),
        r * np.sin(theta) * np.sin(phi),
        r * np.cos(theta),
    ]
)

plotter = pv.Plotter(notebook=True, shape=(1, 3), window_size=(1024, 200))
for i in (1, 2, 3):
    plotter.subplot(0, i-1)
    plotter.add_text(f'{i}s', font_size=10, color='k')
    psi = beregn_orbital(i, 0, 0, r, theta, phi)
    orbital = np.abs(psi)**2
    X, Y, Z = xyz
    grid = pv.StructuredGrid(X, Y, Z)
    plotter.add_mesh(grid, scalars=(r**2 * orbital).T, show_scalar_bar=False)
    plotter.view_xz(-1)
plotter.show(jupyter_backend='static')

Fra figuren over ser vi at området med høyest sannsynlighet (den gule fargen) flytter seg lengre ut når vi øker hovedkvantetallet. La oss sjekke dette ved å plotte radiell del av bølgefunksjonen for det samme området:

In [ ]:
# La oss undersøke radiell del for 1s, 2s og 2p:
r = np.linspace(0, 30, 1000)
fig, axi = plt.subplots(constrained_layout=True)
psi_r_1s = radiell(1, 0, r)
psi_r_2s = radiell(2, 0, r)
psi_r_3s = radiell(3, 0, r)

axi.plot(r, r**2 * abs(psi_r_1s)**2, label='1s', lw=3, alpha=0.8)
axi.plot(r, r**2 * abs(psi_r_2s)**2, label='2s', lw=3, alpha=0.8)
axi.plot(r, r**2 * abs(psi_r_3s)**2, label='3s', lw=3, alpha=0.8)
axi.set(xlabel='$\hat{r}$', ylabel='$\hat{r}^2 \\times R_{n\ell}(\hat{r})$', title='Radiell avhengighet')
axi.legend();

I figuren over ser vi at:
* Den høyeste toppen (området med størst sannsynlighet) flytter seg lenger ut når hovedkvantetallet øker.
* Vi får flere nullpunkter når hovedkvanetallet øker.

Begge disse observasjonene stemmer med det vi så i 3D-plottet av 1s, 2s og 3s over.

Til slutt, la oss plotte 2p-orbitalet og studere fortegnet til bølgefunksjonen:

In [ ]:
plotter = pv.Plotter()

r = np.arange(0.01, 10.0, 0.2)
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(np.pi, 2*np.pi, 100)  # 180 <= phi <= 360, dette blir en halvkule
r, theta, phi = np.meshgrid(r, theta, phi)

# Beregn overflater:
xyz = np.array(
    [
        r * np.sin(theta) * np.cos(phi),
        r * np.sin(theta) * np.sin(phi),
        r * np.cos(theta),
    ]
)
psi = beregn_orbital(2, 1, 1, r, theta, phi)
orbital = np.abs(psi)**2
X, Y, Z = xyz
grid = pv.StructuredGrid(X, Y, Z)
fortegn = 2. * (psi - psi.min()) / np.ptp(psi) - 1
plotter.add_mesh(grid, scalars=fortegn.T,
                 show_scalar_bar=True, clim=[-1, 1],
                 cmap='Spectral',
                 scalar_bar_args={'title': 'Skalert fortegn', 'color': 'k'})

plotter.show(jupyter_backend='ipygany')

Fra figuren over ser vi at vi har to "lober" (blå og rød) som har forskjellig fortegn. Mellom lobene ser vi at vi har et område der bølgefunksjonen er 0 (der vil vi da ikke finne noen elektroner).

## Til slutt
Hvis du har blitt mer nysgjerrig på kvantemekanikk og hvordan vi kan bruke dette til å beskrive atomer og molekyler, så kan jeg anbefale faget [KJ1041 - Fysikalsk kjemi: molekylær struktur](https://www.ntnu.no/studier/emner/KJ1041/#tab=omEmnet).

### Appendix: Eksempler på matematiske uttrykk for orbitaler
Hvis vi ønsker å se hvordan bølgefunksjonene faktisk ser ut, så må vi sette inn verdier for $n$, $\ell$ og $m$,
og slå opp i formersamlinger for å se hvordan de korresponderende Laguerre-polynomene og de sfæriske harmoniske funsjonene ser ut.

#### 1s- og 2s-orbitaler
For s-orbitaler har vi $n=1,2,3,\ldots$, $\ell=0$, $m=0$. La oss se på de 2 første, 1s ($n=1$) og 2s ($n=2$).
Fra [Wikipedia-siden om Laguerre polynomials](https://en.wikipedia.org/wiki/Laguerre_polynomials) kan vi finne:

* for $n=1$: $L_{0}^{(1)}(\hat{r}) =1 $,
* for $n=2$: $L_{1}^{(1)}(\hat{r}) = 2 - \hat{r}$,

og da blir den radielle avhengigheten:
* for $n=1$: $$R_{1\ell=0}(r) = 2  e^{-\hat{r}/2}  L_{0}^{(1)}({\hat{r}}) = 2  \text{e}^{-\hat{r}/2}$$
* for $n=2$: $$R_{2\ell=0}(r) = \frac{1}{2 \sqrt{2}}e^{-\hat{r}/2} L_{1}^{(1)} =   \frac{1}{2 \sqrt{2}} \text{e}^{-\hat{r}/2} (2-\hat{r})$$

Fra [Wikipedia-siden om spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics)
finner vi $Y_{0,0} = \left(\frac{1}{4 \pi}\right)^{1/2} = \frac{1}{2 \sqrt{\pi}}$.

Setter vi sammen alt ($\psi _{n, 0, 0}=R_{n, 0}(r)\,Y_{0,0}(\theta ,\phi )$) får vi:

$$\begin{eqnarray}
\text{1s} &=& \frac{1}{\sqrt{\pi}}  \text{e}^{-\hat{r}/2}, \\
\text{2s} &=& \frac{1}{4\sqrt{2\pi}} (2-\hat{r}) \text{e}^{-\hat{r}/2}, \\
 \end{eqnarray}$$


#### 2p-orbitaler
For 2p-orbitaler har vi $n=2$, $l=1$, $m=-1, 0, 1$.
Fra [Wikipedia-siden om Laguerre polynomials](https://en.wikipedia.org/wiki/Laguerre_polynomials) kan vi finne $L_0^{(3)} = 1$,
og da blir den radielle avhengigheten:

$$R_{2, 1} = \frac{1}{2 \sqrt{6}} \hat{r} \text{e}^{-\hat{r}/2} $$

og fra [Wikipedia-siden om spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics) kan vi finne,
* for $\ell=1$ og $m=0$: $Y_{1, 0} = \left(\frac{3}{4 \pi}\right)^{1/2} \cos \theta$
* for $\ell=1$ og $m=-1$: $Y_{1, -1} = \frac{1}{\sqrt{2}} \left(\frac{3}{4 \pi}\right)^{1/2}  \sin \theta \, \text{e}^{-\text{i} \phi}$
* for $\ell=1$ og $m=1$: $Y_{1, 1} =  -\frac{1}{\sqrt{2}} \left(\frac{3}{4 \pi}\right)^{1/2}  \sin \theta \, \text{e}^{\text{i} \phi} $

La oss sette dette sammen:
* $n=2$, $\ell=1$, $m=0$:
  $$\text{2p}_{\text{z}} = \frac{1}{4 \sqrt{2\pi}} \hat{r} \, \text{e}^{-\hat{r}/2}  \cos \theta$$
* $n=2$, $\ell=1$, $m=-1$, $m=+1$:
  $$\begin{eqnarray}
  \text{2p}_{\text{x}} &=& \frac{1}{\sqrt{2}} (\psi_{2, 1, -1} - \psi_{2, 1, 1}) = 
  \frac{1}{4 \sqrt{2 \pi}} \hat{r} \, \text{e}^{-\hat{r}/2}  \sin \theta \cos \phi ,\\
  \text{2p}_{\text{y}} &=& \frac{\text{i}}{\sqrt{2}} (\psi_{2, 1, -1} + \psi_{2, 1, 1}) =
  \frac{1}{4 \sqrt{2 \pi}} \hat{r} \, \text{e}^{-\hat{r}/2}  \sin \theta \sin \phi .
  \end{eqnarray} $$